In [159]:
import pandas as pd
import numpy as np


#### Global backtracking
* https://gtuckerkellogg.github.io/pairwise/demo/    
* https://github.com/ZhangQiuxue/Rosalind/blob/master/Textbook_05E.py


In [153]:
### https://www.youtube.com/watch?v=DuikFLPt8WQ

#### Recursive solution
def LCSM(x,y,i,j):
    if i <= 0 or j <= 0:
        return 0
    else:
        if x[i-1] == y[j-1]:
            return 1 + LCSM(x,y,i-1,j-1)
        else:
            return max(LCSM(x,y,i,j-1),LCSM(x,y,i-1,j))
        
        
def LCSM_params(x,y,i,j,match=1,indel=0,mismatch=0):
    if i <= 0 or j <= 0:
        return 0
    else:
        if x[i-1] == y[j-1]:
            return LCSM(x,y,i-1,j-1) + match
        else:
            return max(LCSM(x,y,i,j-1) + indel, #gap
                       LCSM(x,y,i-1,j) + indel, #gap
                       LCSM(x,y,i-1,j-1) + mismatch) #mismatch
        
def LCSM_memo(x,y,match=1,indel=0,mismatch=0):
    m = len(x) + 1
    n = len(y) + 1
    memo = np.zeros((m,n))
    for i in range(0,m):
        for j in range(0,n):
            if i == 0 or j == 0:
                memo[i][j] == 0
            else:
                #print(x[i-1],y[j-1])
                if x[i-1] == y[j-1]:
                    memo[i][j] = memo[i-1][j-1] + match
                else:
                    memo[i][j] = max(memo[i-1][j] + indel, #gap
                                     memo[i][j-1] + indel, #gap
                                     memo[i-1][j-1] + mismatch) #mismatch
    return memo



def LCSM_lin(x,y,match=1,indel=0,mismatch=0):
    """
    More efficient. Time is O(m*n) but space is O(n)
    """
    m = len(x) + 1
    n = len(y)  + 1

    memo = [0 for _ in range(0,n)]
    
    for i in range(1,m):
        prev = 0
        for j in range(1,n):
            temp = memo[j]
        
            #print(x[i-1],y[j-1])
            if x[i-1] == y[j-1]:
                memo[j] = prev + match

            else:
                memo[j] = max(memo[j] + indel, #gap
                                 memo[j-1] + indel) #gap
                                 #memo[i-1][j-1] + indel) #mismatch
            prev = temp

    return memo[n-1]

In [5]:
x='JAVAAID'
y='JAVAID'
i = len(x) 
j = len(y) 
LCSM(x,y,i,j)

6

In [8]:
x='JAVAAID'
y='JAVAID'
i = len(x) 
j = len(y) 
LCSM_params(x,y,i,j)

6

In [29]:
x='JAVAAID'
y='JAVAID'
LCSM_memo(x,y)

array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 1.],
       [0., 1., 2., 2., 2., 2., 2.],
       [0., 1., 2., 3., 3., 3., 3.],
       [0., 1., 2., 3., 4., 4., 4.],
       [0., 1., 2., 3., 4., 4., 4.],
       [0., 1., 2., 3., 4., 5., 5.],
       [0., 1., 2., 3., 4., 5., 6.]])

In [155]:
LCSM_lin(x,y)

6

In [126]:
##https://www.youtube.com/watch?v=MiqoA-yF-0M
def edit_distance(x,y):
    m = len(x) + 1
    n = len(y) + 1
    memo = np.zeros((m,n))
    ## initialize
    memo[0] = [0] + list(np.cumsum([1 for i in y]))
    memo[:,0] = [0] + list(np.cumsum([1 for i in x]))
    
    for i in range(1,m):
        for j in range(1,n):
            if i == 0 or j == 0:
                memo[i][j] == 0
            else:
                #print(x[i-1],y[j-1])
                if x[i-1] == y[j-1]:
                    memo[i][j] = memo[i-1][j-1]
                else:
                    memo[i][j] = min(memo[i-1][j] + 1, #gap
                                     memo[i][j-1] + 1, #gap
                                     memo[i-1][j-1] + 1) #mismatch
    return memo[m-1,n-1]

In [127]:
edit_distance('saturday','sundays')

4.0

---

### My approach

In [ ]:
residue='A'
y='ATTCAA'

indel = 0
mismatch = 0
match = 1

score = list(np.cumsum([indel for _ in y]))
nullScore = list(np.cumsum([0] + [indel for _ in x]))

In [173]:
def linspacAlignment(residue,y,updatedScore,nullScore,match=1,indel=0,mismatch=0,j=0,verbose=False):    
    """
    Alignment in linear space
    """
    n = len(y)

    updatedScore = [nullScore[j]] + updatedScore
    prevScore = updatedScore.copy()

    #(prevScore,updatedScore)

    for i in range(1,n+1):
        if residue == y[i-1]:
            if verbose: print(i, 'match',y[i-1], prevScore,updatedScore)
            updatedScore[i] = prevScore[i-1] + match
        else:
            if verbose: print(i,'no match', y[i-1], prevScore,updatedScore,'choose',
                                  updatedScore[i-1],
                                  updatedScore[i],prevScore[i-1])
            updatedScore[i] = max(updatedScore[i-1] + indel, #Up
                                  updatedScore[i] + indel, #Left
                                  prevScore[i-1] + mismatch) #Mismatch
        if verbose: print(updatedScore,"\n----")
    return updatedScore[1:]

def residueSplit(y):
    return [_ for _ in y]

def getScoring():
    '''
    Read scoring matrix
    '''
    with open('blosum62.txt') as matrix_file:
        matrix = matrix_file.read()
        lines = matrix.strip().split('\n')

    blosum={}
    cols = lines[0].split()

    for row in lines[1:]:       
        idx = row[0]
        vals = row[1:].split()
        vals = [int(_) for _ in vals]
        #print(idx,vals)
        blosum[idx]=vals

    blosum = pd.DataFrame.from_dict(blosum,orient='index',columns=cols)
    return blosum

def scorematch(a,b,scoringMatrix):
    '''
    Get the value of a score between two matches
    '''
    return scoringMatrix.loc[a][b]

#### Matrix for nucleotide match, mismatch, indels with penalty 0.
residueScores = pd.DataFrame([[1 if i ==j else 0 for i in 'ACTG'] for j in 'ACTG'])
residueScores.index=['A','C','T','G']
residueScores.columns=['A','C','T','G']
residueScores

#### Blosum62
blosum = getScoring()
blosum.head()

,A,R,N,D,C,Q,E,G,H,I,...,P,S,T,W,Y,V,B,Z,X,*
A,4,-1,-2,-2,0,-1,-1,0,-2,-1,...,-1,1,0,-3,-2,0,-2,-1,0,-4
R,-1,5,0,-2,-3,1,0,-2,0,-3,...,-2,-1,-1,-3,-2,-3,-1,0,-1,-4
N,-2,0,6,1,-3,0,0,0,1,-3,...,-2,1,0,-4,-2,-3,3,0,-1,-4
D,-2,-2,1,6,-3,0,2,-1,-1,-3,...,-1,0,-1,-4,-3,-3,4,1,-1,-4
C,0,-3,-3,-3,9,-3,-4,-3,-3,-1,...,-3,-1,-1,-2,-2,-1,-3,-3,-2,-4


In [147]:
x='ACGGAA'
y='ATTCAA'

indel = 0
mismatch = 0
match = 1

###
score = list(np.cumsum([indel for _ in y]))
nullScore = list(np.cumsum([0] + [indel for _ in x]))

dScores = {0:score}
for i in range(1,len(x)+1):
    dScores[i] = linspacAlignment(x[i-1],y,dScores[i-1],nullScore,verbose=False,j=i-1)

In [148]:
ScoringMat = pd.DataFrame.from_dict(dScores.values(),orient='columns').T
ScoringMat.columns=residueSplit("-" + y)
ScoringMat

,-,A,T,T,C,A,A
0,0,1,1,1,1,1,1
1,0,1,1,1,1,1,1
2,0,1,1,1,1,1,1
3,0,1,2,2,2,2,2
4,0,1,2,2,2,3,3
5,0,1,2,2,2,3,4


In [161]:
def linspacAlignment_scoreMat(residue,y,updatedScore,nullScore,scoringMatrix,indel=0,j=0,verbose=False):    
    """
    Alignment in linear space
    Add scoring matrix capability
    """
    n = len(y)

    updatedScore = [nullScore[j]] + updatedScore
    prevScore = updatedScore.copy()

    #(prevScore,updatedScore)

    for i in range(1,n+1):
        if residue == y[i-1]:
            if verbose: print(i, 'match',y[i-1], prevScore,updatedScore)
            updatedScore[i] = prevScore[i-1] + scorematch(residue,y[i-1],scoringMatrix) 
        else:
            if verbose: print(i,'no match', y[i-1], prevScore,updatedScore,'choose',
                                  updatedScore[i-1],
                                  updatedScore[i],prevScore[i-1])
            updatedScore[i] = max(updatedScore[i-1] + indel, #Up
                                  updatedScore[i] + indel, #Left
                                  prevScore[i-1] + scorematch(residue,y[i-1],scoringMatrix)) #Mismatch
        if verbose: print(updatedScore,"\n----")
    return updatedScore[1:]


In [162]:
%time
### Using the aligner with a scoring matrix
x='ACGGAA'
y='ATTCAA'

indel = 0
mismatch = 0
match =1

###
score = list(np.cumsum([indel for _ in y]))
nullScore = list(np.cumsum([0] + [indel for _ in x]))

dScores = {0:score}
for i in range(1,len(x)+1):
    dScores[i] = linspacAlignment_scoreMat(x[i-1],y,dScores[i-1],nullScore,scoringMatrix=residueScores,verbose=False,j=i-1)
    

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 24.8 µs


In [163]:
ScoringMat = pd.DataFrame.from_dict(dScores.values(),orient='columns').T
ScoringMat.columns=residueSplit("-" + y)
ScoringMat

,-,A,T,T,C,A,A
0,0,1,1,1,1,1,1
1,0,1,1,1,1,1,1
2,0,1,1,1,1,1,1
3,0,1,2,2,2,2,2
4,0,1,2,2,2,3,3
5,0,1,2,2,2,3,4


In [ ]:
5

In [164]:
def wrapAligner(x,y,scoringMatrix,indel=0,verbose=False):
    # Initialize scoring 
    score = list(np.cumsum([indel for _ in y]))
    nullScore = list(np.cumsum([0] + [indel for _ in x]))

    dScores = {0:score}
    for i in range(1,len(x)+1):
        dScores[i] = linspacAlignment_scoreMat(x[i-1],y,dScores[i-1],nullScore,scoringMatrix=scoringMatrix,indel=indel,verbose=False,j=i-1)
    tmpMat = pd.DataFrame.from_dict(dScores.values(),orient='columns').T
    return pd.DataFrame(nullScore).T.append(tmpMat).reset_index(drop=True)

In [412]:
%time
x='PLEASANTLY'
y='MEANLY'
aligned = wrapAligner(x,y,blosum,indel=-5)
aligned

CPU times: user 7 µs, sys: 1e+03 ns, total: 8 µs
Wall time: 41 µs


,0,1,2,3,4,5,6,7,8,9,10
0,0,-5,-10,-15,-20,-25,-30,-35,-40,-45,-50
1,-5,-2,-3,-8,-13,-18,-23,-28,-33,-38,-43
2,-10,-6,-5,2,-3,-8,-13,-18,-23,-28,-33
3,-15,-11,-7,-3,6,1,-4,-9,-14,-19,-24
4,-20,-16,-12,-7,1,7,2,2,-3,-8,-13
5,-25,-21,-12,-12,-4,2,6,1,1,1,-4
6,-30,-26,-17,-14,-9,-3,1,4,-1,0,8


In [413]:
%time
x = 'ILYPRQSMICMSFCFWDMWKKDVPVVLMMFLERRQMQSVFSWLVTVKTDCGKGIYNHRKYLGLPTMTAGDWHWIKKQNDPHEWFQGRLETAWLHSTFLYWKYFECDAVKVCMDTFGLFGHCDWDQQIHTCTHENEPAIAFLDLYCRHSPMCDKLYPVWDMACQTCHFHHSWFCRNQEMWMKGDVDDWQWGYHYHTINSAQCNQWFKEICKDMGWDSVFPPRHNCQRHKKCMPALYAGIWMATDHACTFMVRLIYTENIAEWHQVYCYRSMNMFTCGNVCLRCKSWIFVKNYMMAPVVNDPMIEAFYKRCCILGKAWYDMWGICPVERKSHWEIYAKDLLSFESCCSQKKQNCYTDNWGLEYRLFFQSIQMNTDPHYCQTHVCWISAMFPIYSPFYTSGPKEFYMWLQARIDQNMHGHANHYVTSGNWDSVYTPEKRAGVFPVVVPVWYPPQMCNDYIKLTYECERFHVEGTFGCNRWDLGCRRYIIFQCPYCDTMKICYVDQWRSIKEGQFRMSGYPNHGYWFVHDDHTNEWCNQPVLAKFVRSKIVAICKKSQTVFHYAYTPGYNATWPQTNVCERMYGPHDNLLNNQQNVTFWWKMVPNCGMQILISCHNKMKWPTSHYVFMRLKCMHVLMQMEYLDHFTGPGEGDFCRNMQPYMHQDLHWEGSMRAILEYQAEHHRRAFRAELCAQYDQEIILWSGGWGVQDCGFHANYDGSLQVVSGEPCSMWCTTVMQYYADCWEKCMFA'
y = 'ILIPRQQMGCFPFPWHFDFCFWSAHHSLVVPLNPQMQTVFQNRGLDRVTVKTDCHDHRWKWIYNLGLPTMTAGDWHFIKKHVVRANNPHQWFQGRLTTAWLHSTFLYKKTEYCLVRHSNCCHCDWDQIIHTCAFIAFLDLYQRHWPMCDKLYCHFHHSWFCRNQEMSMDWNQWFPWDSVPRANCLEEGALIALYAGIWANSMKRDMKTDHACTVRLIYVCELHAWLKYCYTSINMLCGNVCLRCKSWIFVKLFYMYAPVVNTIEANSPHYYKRCCILGQGICPVERKSHCEIYAKDLLSFESCCSQKQNCYTDNWGLEYRLFFQHIQMECTDPHANRGWTSCQTAKYWHFNLDDRPPKEFYMWLQATPTDLCMYQHCLMFKIVKQNFRKQHGHANPAASTSGNWDSVYTPEKMAYKDWYVSHPPVDMRRNGSKMVPVWYPPGIWHWKQSYKLTYECFFTVPGRFHVEGTFGCNRWDHQPGTRRDRQANHQFQCPYSDTMAIWEHAYTYVDQWRSIKEGQMPMSGYPNHGQWNVHDDHTNEQERSPICNQPVLAKFVRSKNVSNHEICKKSQTVFHWACEAQTNVCERMLNNQHVAVKRNVTFWWQMVPNCLWSCHNKMTWPTRPEQHRLFFVKMRLKCMHEYLDVAPSDFCRNMQAYMHSMRAILEYQADFDLKRRLRAIAPMDLCAQYDQEIILWSGGYIYDQSLQVVSCEGCSYYADCYVKCINVKEKCMFA'
wrapAligner(x,y,blosum,indel=-5)

CPU times: user 32 µs, sys: 3 µs, total: 35 µs
Wall time: 340 µs


KeyboardInterrupt: 

Order doesn't matter:

In [414]:
%time
x='PLEASANTLY'
y='MEANLY'
wrapAligner(y,x,blosum,indel=-5)

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 27.2 µs


,0,1,2,3,4,5,6
0,0,-5,-10,-15,-20,-25,-30
1,-5,-2,-6,-11,-16,-21,-26
2,-10,-3,-5,-7,-12,-12,-17
3,-15,-8,2,-3,-7,-12,-14
4,-20,-13,-3,6,1,-4,-9
5,-25,-18,-8,1,7,2,-3
6,-30,-23,-13,-4,2,6,1
7,-35,-28,-18,-9,2,1,4
8,-40,-33,-23,-14,-3,1,-1
9,-45,-38,-28,-19,-8,1,0


In [415]:
%time
x='PLEASANTLY'
y='MEANLY'
alMat = wrapAligner(x,y,blosum,indel=-5)
alMat

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.25 µs


,0,1,2,3,4,5,6,7,8,9,10
0,0,-5,-10,-15,-20,-25,-30,-35,-40,-45,-50
1,-5,-2,-3,-8,-13,-18,-23,-28,-33,-38,-43
2,-10,-6,-5,2,-3,-8,-13,-18,-23,-28,-33
3,-15,-11,-7,-3,6,1,-4,-9,-14,-19,-24
4,-20,-16,-12,-7,1,7,2,2,-3,-8,-13
5,-25,-21,-12,-12,-4,2,6,1,1,1,-4
6,-30,-26,-17,-14,-9,-3,1,4,-1,0,8


In [416]:
%time
x='PLEASANTLY'
y='MEANLY'
alMat = wrapAligner(x,y,blosum,indel=-5)
print(alMat)

w,v = getAlignment(x,y,alMat,indel=-5,scoringmatrix=blosum)
print(alMat[len(x)][len(y)])
print('---')
print(w)
print(v)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 6.2 µs
    0   1   2   3   4   5   6   7   8   9  10
0   0  -5 -10 -15 -20 -25 -30 -35 -40 -45 -50
1  -5  -2  -3  -8 -13 -18 -23 -28 -33 -38 -43
2 -10  -6  -5   2  -3  -8 -13 -18 -23 -28 -33
3 -15 -11  -7  -3   6   1  -4  -9 -14 -19 -24
4 -20 -16 -12  -7   1   7   2   2  -3  -8 -13
5 -25 -21 -12 -12  -4   2   6   1   1   1  -4
6 -30 -26 -17 -14  -9  -3   1   4  -1   0   8
8
---
PLEASANTLY
-MEA--N-LY


In [314]:
def getAlignment(x,y,alignmentMatrix,indel,scoringmatrix,verbose=False):

    i = len(x)
    j = len(y)

    wOut = ''
    vOut = ''

    while i*j != 0:
        btrack = [alMat.iloc[j-1][i] + indel,  #up
                        alMat.iloc[j][i-1] + indel,   #left
                        alMat.iloc[j-1][i-1] +scorematch(x[i-1],y[j-1],scoringmatrix)]
        mx = np.argmax(btrack) #diag

        if verbose: print(i,j,btrack,mx,btrack[mx])
        if mx == 2:
            wOut += x[i-1]
            vOut += y[j-1]
            i -= 1
            j -= 1
        else:
            if mx == 0:
                wOut += '-'
                vOut += y[j-1]
                j -= 1
            else:
                wOut += x[i-1]
                vOut += '-'
                i -= 1
        if verbose: print(wOut[::-1])
        if verbose: print(vOut[::-1])

   
    if i == 0 and j>0:
        vOut = vOut + y[j-1::-1]
        vOut = vOut[::-1]
        wOut = ''.join(['-' for _ in range(j,0,-1)]) + wOut[::-1]
    if j == 0 and i>0:
        wOut = wOut + x[i-1::-1]
        wOut = wOut[::-1]
        vOut = ''.join(['-' for _ in range(i,0,-1)]) + vOut[::-1]

    return wOut, vOut

### Hirschberg's algorithm

https://ocw.mit.edu/courses/electrical-engineering-and-computer-science/6-096-algorithms-for-computational-biology-spring-2005/lecture-notes/lecture5_newest.pdf

In [315]:
residueScores[residueScores == 1] = 2
residueScores[residueScores == 0] = -1
residueScores

,A,C,T,G
A,2,-1,-1,-1
C,-1,2,-1,-1
T,-1,-1,2,-1
G,-1,-1,-1,2


In [316]:
%time

x='AGTACGCA'
y='TATGC'
alMat = wrapAligner(x,y,residueScores,indel=-2)
print(alMat)

w,v = getAlignment(x,y,alMat,indel=0,scoringmatrix=residueScores)
print('\n~~~\score:')
print(alMat[len(x)][len(y)])
print('~~~\nalginment:')
print(w)
print(v)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 36 µs
    0  1  2  3  4   5   6   7   8
0   0 -2 -4 -6 -8 -10 -12 -14 -16
1  -2 -1 -3 -2 -4  -6  -8 -10 -12
2  -4  0 -2 -4  0  -2  -4  -6  -8
3  -6 -2 -1  0 -2  -1  -3  -5  -7
4  -8 -4  0 -2 -1  -3   1  -1  -3
5 -10 -6 -2 -1 -3   1  -1   3   1

~~~\score:
1
~~~
alginment:
AGTACGCA
--TATGC-


In [330]:
def splitstring(s,point):
    fwd = s[0:point:]
    rev = s[point:][::-1]
    return fwd, rev

def halfstring(s):
    fwd = s[0:int(len(s)/2):]
    rev = s[int(len(s)/2):][::-1]
    return fwd, rev

,A,C,T,G
A,2.0,1.0,1.0,1.0
C,1.0,2.0,1.0,1.0
T,1.0,1.0,2.0,1.0
G,1.0,1.0,1.0,2.0


In [377]:
def createScorMat(match = 1,mismatch = 0):
    """
    Create scoring matrix for nucleotides with a given match and mismatch values
    """
    nts = ['A','C','T','G']

    scoringMatrix = np.zeros((len(nts),len(nts)),)
    np.fill_diagonal(scoringMatrix,2)
    scoringMatrix
    scoringMatrix[scoringMatrix == 0] = mismatch
    scoringMatrix = pd.DataFrame(scoringMatrix,index=nts,columns=nts)
    return scoringMatrix

In [380]:
x
scoringMatrix = createScorMat(2,1)


n = len(y)

score = list(np.cumsum([indel for _ in y]))
nullScore = list(np.cumsum([0] + [indel for _ in x]))

score,nullScore

([0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0])

In [346]:
def linspace(residue,y,updatedScore,nullScore,scoringMatrix,indel=0,j=0,verbose=False):    
    """
    Alignment in linear space
    Add scoring matrix capability
    """
    n = len(y)

    updatedScore = [nullScore[j]] + updatedScore
    prevScore = updatedScore.copy()

    #(prevScore,updatedScore)

    for i in range(1,n+1):
        if residue == y[i-1]:
            if verbose: print(i, 'match',y[i-1], prevScore,updatedScore)
            updatedScore[i] = prevScore[i-1] + scorematch(residue,y[i-1],scoringMatrix) 
        else:
            if verbose: print(i,'no match', y[i-1], prevScore,updatedScore,'choose',
                                  updatedScore[i-1],
                                  updatedScore[i],prevScore[i-1])
            updatedScore[i] = max(updatedScore[i-1] + indel, #Up
                                  updatedScore[i] + indel, #Left
                                  prevScore[i-1] + scorematch(residue,y[i-1],scoringMatrix)) #Mismatch
        if verbose: print(updatedScore,"\n----")
    return updatedScore[1:]


def align_linear(x,y,scoringMatrix,indel=0,verbose=False):
    # Initialize scoring 
    score = list(np.cumsum([indel for _ in y]))
    nullScore = list(np.cumsum([0] + [indel for _ in x]))
    for i in range(1,len(x)+1):
        prev = score.copy()
        score = linspace(x[i-1],y,prev,nullScore,scoringMatrix=scoringMatrix,indel=indel,verbose=False,j=i-1)
    return score

In [331]:
def hirsh(x,y,scoringmatrix,indel):
    fwd, rev = halfstring(x)
    fwdMat = wrapAligner(y,fwdX,scoringmatrix,indel)

('AGTA', 'ACGC')

In [347]:
align_linear(x,y,residueScores,indel=-2,verbose=False)

IndexError: list index out of range

In [319]:
fwdX = x[0:int(len(x)/2):]
print(x,fwdX)
fwdMat = wrapAligner(y,fwdX,residueScores,indel=-2)
fwdMat


AGTACGCA AGTA


,0,1,2,3,4,5
0,0,-2,-4,-6,-8,-10
1,-2,-1,0,-2,-4,-6
2,-4,-3,-2,-1,0,-2
3,-6,-2,-4,0,-2,-1
4,-8,-4,0,-2,-1,-3


In [320]:
revX = x[int(len(x)/2):][::-1]
print(x,revX)
revMat = wrapAligner(y[::-1],revX,residueScores,indel=-2)
revMat

AGTACGCA ACGC


,0,1,2,3,4,5
0,0,-2,-4,-6,-8,-10
1,-2,-1,-3,-5,-4,-6
2,-4,0,-2,-4,-6,-5
3,-6,-2,2,0,-2,-4
4,-8,-4,0,1,-1,-3


In [323]:
## Where to partition: max of both sums
a=revMat.iloc[len(revMat)-1][::-1].reset_index(drop=True)
b=fwdMat.iloc[len(fwdMat)-1]
partitionY = np.argmax(a+b)
(a,b,a+b,partitionY)

(0   -3
 1   -1
 2    1
 3    0
 4   -4
 5   -8
 Name: 4, dtype: int64,
 0   -8
 1   -4
 2    0
 3   -2
 4   -1
 5   -3
 Name: 4, dtype: int64,
 0   -11
 1    -5
 2     1
 3    -2
 4    -5
 5   -11
 Name: 4, dtype: int64,
 2)

In [332]:
splitstring(y,partitionY)


('TA', 'CGT')

In [333]:
halfstring(fwdX)

('AG', 'AT')

5   -3
4   -1
3    1
2    0
1   -4
0   -8
Name: 4, dtype: int64

---

From github

In [136]:
## From https://github.com/ZhangQiuxue/Rosalind/blob/master/Textbook_05E.py
def global_alignment(v, w, scoring_matrix, sigma):
    '''Returns the global alignment of v and w subject to the given scoring matrix and indel penalty sigma.'''
    # Initialize the matrices.
    S = [[0]*(len(w)+1) for _ in range(len(v)+1)]
    backtrack = [[0]*(len(w)+1) for _ in range(len(v)+1)]

    # Initialize the edges with the given penalties.
    for i in range(1, len(v)+1):
        S[i][0] = -i*sigma
    for j in range(1, len(w)+1):
        S[0][j] = -j*sigma

    # Fill in the Score and Backtrack matrices.
    for i in range(1, len(v)+1):
        for j in range(1, len(w)+1):
            scores = [S[i-1][j] - sigma, S[i][j-1] - sigma, S[i-1][j-1] + blosum.loc[v[i-1]][w[j-1]]]
            S[i][j] = max(scores)
            backtrack[i][j] = scores.index(S[i][j])

    # Quick lambda function to insert indels.
    insert_indel = lambda word, i: word[:i] + '-' + word[i:]

    # Initialize the aligned strings as the input strings.
    v_aligned, w_aligned = v, w

    # Get the position of the highest scoring cell in the matrix and the high score.
    i, j = len(v), len(w)
    max_score = str(S[i][j])

    # Backtrack to the edge of the matrix starting at the highest scoring cell.
    while i*j != 0:
        if backtrack[i][j] == 0:
            i -= 1
            w_aligned = insert_indel(w_aligned, j)
        elif backtrack[i][j] == 1:
            j -= 1
            v_aligned = insert_indel(v_aligned, i)
        else:
            i -= 1
            j -= 1

    # Prepend the necessary preceeding indels to get to (0,0).
    for _ in range(i):
        w_aligned = insert_indel(w_aligned, 0)
    for _ in range(j):
        v_aligned = insert_indel(v_aligned, 0)

    return max_score, v_aligned, w_aligned,scores,S,backtrack

In [126]:
%time
v='PLEASANTLY'
w='MEANLY'
global_alignment(v, w, blosum, 5)

CPU times: user 19 µs, sys: 5 µs, total: 24 µs
Wall time: 1.65 ms


('8', 'PLEASANTLY', '-MEA--N-LY')

In [128]:
%time
v='ILYPRQSMICMSFCFWDMWKKDVPVVLMMFLERRQMQSVFSWLVTVKTDCGKGIYNHRKYLGLPTMTAGDWHWIKKQNDPHEWFQGRLETAWLHSTFLYWKYFECDAVKVCMDTFGLFGHCDWDQQIHTCTHENEPAIAFLDLYCRHSPMCDKLYPVWDMACQTCHFHHSWFCRNQEMWMKGDVDDWQWGYHYHTINSAQCNQWFKEICKDMGWDSVFPPRHNCQRHKKCMPALYAGIWMATDHACTFMVRLIYTENIAEWHQVYCYRSMNMFTCGNVCLRCKSWIFVKNYMMAPVVNDPMIEAFYKRCCILGKAWYDMWGICPVERKSHWEIYAKDLLSFESCCSQKKQNCYTDNWGLEYRLFFQSIQMNTDPHYCQTHVCWISAMFPIYSPFYTSGPKEFYMWLQARIDQNMHGHANHYVTSGNWDSVYTPEKRAGVFPVVVPVWYPPQMCNDYIKLTYECERFHVEGTFGCNRWDLGCRRYIIFQCPYCDTMKICYVDQWRSIKEGQFRMSGYPNHGYWFVHDDHTNEWCNQPVLAKFVRSKIVAICKKSQTVFHYAYTPGYNATWPQTNVCERMYGPHDNLLNNQQNVTFWWKMVPNCGMQILISCHNKMKWPTSHYVFMRLKCMHVLMQMEYLDHFTGPGEGDFCRNMQPYMHQDLHWEGSMRAILEYQAEHHRRAFRAELCAQYDQEIILWSGGWGVQDCGFHANYDGSLQVVSGEPCSMWCTTVMQYYADCWEKCMFA'
w='ILIPRQQMGCFPFPWHFDFCFWSAHHSLVVPLNPQMQTVFQNRGLDRVTVKTDCHDHRWKWIYNLGLPTMTAGDWHFIKKHVVRANNPHQWFQGRLTTAWLHSTFLYKKTEYCLVRHSNCCHCDWDQIIHTCAFIAFLDLYQRHWPMCDKLYCHFHHSWFCRNQEMSMDWNQWFPWDSVPRANCLEEGALIALYAGIWANSMKRDMKTDHACTVRLIYVCELHAWLKYCYTSINMLCGNVCLRCKSWIFVKLFYMYAPVVNTIEANSPHYYKRCCILGQGICPVERKSHCEIYAKDLLSFESCCSQKQNCYTDNWGLEYRLFFQHIQMECTDPHANRGWTSCQTAKYWHFNLDDRPPKEFYMWLQATPTDLCMYQHCLMFKIVKQNFRKQHGHANPAASTSGNWDSVYTPEKMAYKDWYVSHPPVDMRRNGSKMVPVWYPPGIWHWKQSYKLTYECFFTVPGRFHVEGTFGCNRWDHQPGTRRDRQANHQFQCPYSDTMAIWEHAYTYVDQWRSIKEGQMPMSGYPNHGQWNVHDDHTNEQERSPICNQPVLAKFVRSKNVSNHEICKKSQTVFHWACEAQTNVCERMLNNQHVAVKRNVTFWWQMVPNCLWSCHNKMTWPTRPEQHRLFFVKMRLKCMHEYLDVAPSDFCRNMQAYMHSMRAILEYQADFDLKRRLRAIAPMDLCAQYDQEIILWSGGYIYDQSLQVVSCEGCSYYADCYVKCINVKEKCMFA'
global_alignment(v, w, blosum, 5)

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 41 µs


('1555',
 'ILYPRQSMICMSFCF-WDM--WKKDVPVVLMMFLERRQMQSVF-S-WL--VTVKTDCGKGIYNHR-K--Y-LGLPTMTAGDWHWIKK---Q-NDPHEWFQGRLETAWLHSTFLYWKYFE-CDAVKVCMDTFGLFGHCDWDQQIHTCTHENEPAIAFLDLYCRHSPMCDKLYPVWDMACQTCHFHHSWFCRNQEMWMKGDVDDWQWGYHYHTINSAQCNQWFKEICKDMGWDSVFPPRHNCQRHKKCMPALYAGIW---MA----TDHACTFMVRLIYTENIAEWHQVYCYRSMNMFTCGNVCLRCKSWIFVKN-YMMAPVVNDPMIEA--FYKRCCILGKAWYDMWGICPVERKSHWEIYAKDLLSFESCCSQKKQNCYTDNWGLEYRLFFQSIQMN-TDPH----Y--CQTHVCW-ISAMF-PIYSPFYT--SG-PKEFYMW---LQARI-DQNM---HGHANHYV-TSGNWDSVYTPEKRA--G--V-FP-V-V-------VPVWYPPQMCN--D-YIKLTYEC--E---RFHVEGTFGCNRWD-L-GCRR--YII--FQCPYCDTMKI---CY--VDQWRSIKEGQFRMSGYPNHGYWFVHDDHTNEW-----CNQPVLAKFVRSKIVA---ICKKSQTVFHYAYTPGYNATWPQTNVCERMYGPHDNLLNNQQNVTFWWKMVPNCGMQILISCHNKMKWPT--S-HYVF---MRLKCMHVLMQMEYLDHFTGPGEGDFCRNMQPYMHQDLHWEGSMRAILEYQAEHH-RRAFRA----ELCAQYDQEIILWSGGWGVQDCGFHANYDGSLQVVSGEPCSMWCTTVMQYYADCWEKCMFA',
 'ILIPRQQMGCFPFPWHFDFCFWSAHHSLVVP--LNP-QMQTVFQNRGLDRVTVKTDC----HDHRWKWIYNLGLPTMTAGDWHFIKKHVVRANNPHQWFQGRLTTAWLHSTFLY-KKTEYC-LVR---HS-NCC-HCDWDQ